In [1]:
# importações
from utils import *
from utils_backtest import *
from models import *
from config import *

# 1. Aplicabilidade do Modelo 2 no Mercado Real

In [2]:
# Lista de ações
stock_list = ['PETR4', 'VALE3', 'BBDC4','ITUB4']

# Carrega os dados de todas as ações e calcula os indicadores e salva em um dicionario
all_stock_data = load_stocks_data(stock_list)

Carregando dados para: PETR4
Carregando dados para: VALE3
Carregando dados para: BBDC4
Carregando dados para: ITUB4


## 2. Processar o Treino do Modelo para Salvar os artefatos

In [3]:
# Processando o Modelo 2 para Salvar os Artefatos (modelo e scalers)

name_model = model_2 # Modelo 2
num_combination = 5 # Combinação 5

'''
7 combinações de dados disponíveis: 
     1. Stock Data
     2. Stock Data + Google News
     3. Stock Data + Twitter 
     4. Stock Data + (IFR + MMS + MME) 
     5. Google News + Twitter + IFR + MMS 
     6. Google News + Twitter + IFR + MME
     7. Google News + Twitter + IFR + MME + MMS 
'''

result = process_stocks_and_save_metrics(all_stock_data, num_combination, name_model, save_artifacts=True) # Salva os artefatos

model_2: Processando PETR4 | Combinação 5...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 0.0576 - mae: 0.1759 - mape: 3116.7078 - mse: 0.0576 - val_loss: 6.3071e-04 - val_mae: 0.0173 - val_mape: 4.3974 - val_mse: 6.3071e-04
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 4.3347e-04 - mae: 0.0158 - mape: 708.1550 - mse: 4.3347e-04 - val_loss: 5.2812e-04 - val_mae: 0.0164 - val_mape: 4.0572 - val_mse: 5.2812e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.8904e-04 - mae: 0.0123 - mape: 384.5552 - mse: 2.8904e-04 - val_loss: 4.6129e-04 - val_mae: 0.0142 - val_mape: 3.6171 - val_mse: 4.6129e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 2.8992e-04 - mae: 0.0124 - mape: 468.3375 - mse: 2.8992e-04 - val_loss: 5.1167e-04 - val_mae: 0.0164 - val_mape: 3.9993 - val_mse: 5.1167e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.8746e-04 - mae: 0.0122 - mape: 360.6918 - mse: 2.8746e-04 - val_loss: 6.7671e-04 - val_mae: 0.021

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
model_2: Processando VALE3 | Combinação 5...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0826 - mae: 0.2087 - mape: 125792.4844 - mse: 0.0826 - val_loss: 0.0088 - val_mae: 0.0706 - val_mape: 6.0364 - val_mse: 0.0088
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 4.5484e-04 - mae: 0.0162 - mape: 14444.9473 - mse: 4.5484e-04 - val_loss: 0.0026 - val_mae: 0.0387 - val_mape: 3.6592 - val_mse: 0.0026
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 3.7987e-04 - mae: 0.0146 - mape: 10486.3135 - mse: 3.7987e-04 - val_loss: 0.0024 - val_mae: 0.0389 - val_mape: 3.7063 - val_mse: 0.0024
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 4.0327e-04 - mae: 0.0148 - mape: 7288.1172 - mse: 4.0327e-04 - val_loss: 0.0032 - val_mae: 0.0435 - val_mape: 3.9233 - val_mse: 0.0032
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 3.8045e-04 - mae: 0.0146 - mape: 6509.0054 - mse: 3.8045e-04 - val_loss: 0.0037 - va

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step
model_2: Processando BBDC4 | Combinação 5...
Epoch 1/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 7s 32ms/step - loss: 0.0596 - mae: 0.1699 - mape: 11898.5176 - mse: 0.0596 - val_loss: 0.0014 - val_mae: 0.0263 - val_mape: 5.0750 - val_mse: 0.0014
Epoch 2/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 6.2821e-04 - mae: 0.0179 - mape: 2515.5452 - mse: 6.2821e-04 - val_loss: 8.5527e-04 - val_mae: 0.0207 - val_mape: 4.1109 - val_mse: 8.5527e-04
Epoch 3/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 24ms/step - loss: 5.0794e-04 - mae: 0.0155 - mape: 269.8908 - mse: 5.0794e-04 - val_loss: 7.8478e-04 - val_mae: 0.0195 - val_mape: 3.8878 - val_mse: 7.8478e-04
Epoch 4/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - loss: 5.3622e-04 - mae: 0.0162 - mape: 300.1944 - mse: 5.3622e-04 - val_loss: 7.6561e-04 - val_mae: 0.0192 - val_mape: 3.8347 - val_mse: 7.6561e-04
Epoch 5/50
88/88 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 5.2475e-04 - mae: 0.0161 - mape: 459.0729 - mse: 5.2475e-04 - val

22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
model_2: Processando ITUB4 | Combinação 5...
Epoch 1/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 6s 25ms/step - loss: 0.0634 - mae: 0.1786 - mape: 133815.6719 - mse: 0.0634 - val_loss: 0.0012 - val_mae: 0.0253 - val_mape: 4.1706 - val_mse: 0.0012
Epoch 2/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - loss: 3.8590e-04 - mae: 0.0144 - mape: 52284.1562 - mse: 3.8590e-04 - val_loss: 6.9876e-04 - val_mae: 0.0184 - val_mape: 3.1217 - val_mse: 6.9876e-04
Epoch 3/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 2.9085e-04 - mae: 0.0127 - mape: 42058.8203 - mse: 2.9085e-04 - val_loss: 6.5796e-04 - val_mae: 0.0178 - val_mape: 3.0221 - val_mse: 6.5796e-04
Epoch 4/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 2.8883e-04 - mae: 0.0127 - mape: 36857.6094 - mse: 2.8883e-04 - val_loss: 6.3830e-04 - val_mae: 0.0175 - val_mape: 2.9704 - val_mse: 6.3830e-04
Epoch 5/50
90/90 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - loss: 2.7538e-04 - mae: 0.0123 - mape: 33331.7031 - mse: 2.7538e-

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


## 3. Processar o Backtest com Modelo salvo

In [6]:
# Execute apenas se pulou a etapa anterior e deseja carregar os artefatos

name_model = 'model_2' # Modelo 2
num_combination = 5 # Combinação 5

In [ ]:
results, signals = process_stocks(stock_list, all_stock_data, name_model, 5) # num_combination = 5

Processando PETR4...


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step
PETR4 processado!
Processando VALE3...


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
VALE3 processado!
Processando BBDC4...


22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
BBDC4 processado!
Processando ITUB4...


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step
ITUB4 processado!


In [ ]:
plot_all_results(results, signals, name_model, num_combination)

In [10]:
# Combinação 5: Google News + Twitter + IFR + MMS
final_results_df = calculate_final_results(results, signals)
final_results_df

,Ativo,Estratégia (Modelo 2) (R$),Buy & Hold (R$),Retorno Estratégia (%),Retorno Buy & Hold (%)
0,PETR4,R$ 13426.86,R$ 10739.39,34.27%,7.39%
1,VALE3,R$ 13114.02,R$ 15489.48,31.14%,54.89%
2,BBDC4,R$ 14383.19,R$ 6553.20,43.83%,-34.47%
3,ITUB4,R$ 12249.94,R$ 6668.49,22.50%,-33.32%
